In [1]:
import pandas as pd

df = pd.read_csv("./data/raw/train_cax.csv")
oof = pd.read_csv("./data/result/oof.csv")

dt = pd.read_csv("./data/raw/test_cax.csv")
ds = pd.read_csv("./data/result/LSTM_submit.csv")

dt["label"] = list(ds["label"])
df["pred"] = oof["pred_label"]

In [14]:
from math import floor

df["leak"] = df["GR"]*100
df["leak"] = df["leak"] - df["leak"].apply(floor)
df["leak"] = df["leak"].apply(lambda x: round(x, 11))

for i in range(5):
    for j in range(i+1,5):
        a = set(df[df["label"]==i]["leak"].value_counts().keys())
        b = set(df[df["label"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))

0 1 	 1860 174921 	 0
0 2 	 1860 16100 	 0
0 3 	 1860 155569 	 0
0 4 	 1860 154186 	 0
1 2 	 174921 16100 	 0
1 3 	 174921 155569 	 0
1 4 	 174921 154186 	 0
2 3 	 16100 155569 	 0
2 4 	 16100 154186 	 0
3 4 	 155569 154186 	 0


In [3]:
# 0 1 	 5394 267200 	 0
# 0 2 	 5394 52786 	 0
# 0 3 	 5394 218501 	 0
# 0 4 	 5394 218044 	 0
# 1 2 	 267200 52786 	 0
# 1 3 	 267200 218501 	 0
# 1 4 	 267200 218044 	 0
# 2 3 	 52786 218501 	 0
# 2 4 	 52786 218044 	 0
# 3 4 	 218501 218044 	 0

In [4]:
s = set()

for i in range(5):
    for j in range(i+1,5):
        a = set(df[df["pred"]==i]["leak"].value_counts().keys())
        b = set(df[df["pred"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))
        s |= a&b
        
len(s)

0 1 	 1860 174921 	 0
0 2 	 1860 16100 	 0
0 3 	 1860 155569 	 0
0 4 	 1860 154186 	 0
1 2 	 174921 16100 	 0
1 3 	 174921 155569 	 0
1 4 	 174921 154186 	 0
2 3 	 16100 155569 	 0
2 4 	 16100 154186 	 0
3 4 	 155569 154186 	 0


0

In [5]:
1-len(df[df["label"]!=df["pred"]])/len(df)

1.0

In [6]:
from tqdm import tqdm_notebook as tqdm
d = dict()
for c in tqdm(list(s)):
    k = df[df["leak"]==c]["pred"].value_counts().keys()[0]
    d[c] = k

In [7]:
res = []
for k, v in tqdm(zip(df["leak"], df["pred"]), total=len(df)):
    if k in d:
        res.append(d[k])
    else:
        res.append(v)

In [8]:
df["res"] = res
print(1-len(df[df["label"]!=df["res"]])/len(df))

1.0


In [9]:
dt["leak"] = dt["GR"]*100
dt["leak"] = dt["leak"] - dt["leak"].apply(floor)
dt["leak"] = dt["leak"].apply(lambda x: round(x, 11))

ss = set()

for i in range(5):
    for j in range(i+1,5):
        a = set(dt[dt["label"]==i]["leak"].value_counts().keys())
        b = set(dt[dt["label"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))
        ss |= a&b
        
len(ss)

0 1 	 2805 89233 	 477
0 2 	 2805 8488 	 15
0 3 	 2805 77899 	 220
0 4 	 2805 80403 	 292
1 2 	 89233 8488 	 97
1 3 	 89233 77899 	 191
1 4 	 89233 80403 	 196
2 3 	 8488 77899 	 130
2 4 	 8488 80403 	 127
3 4 	 77899 80403 	 189


1624

In [10]:
dd = dict()
for c in tqdm(list(ss)):
    k = dt[dt["leak"]==c]["label"].value_counts().keys()[0]
    dd[c] = k

In [11]:
res = []
for k, v in tqdm(zip(dt["leak"], dt["label"]), total=len(dt)):
    if k in dd:
        res.append(dd[k])
    else:
        res.append(v)
        
dt["label"] = res

In [12]:
dt[["unique_id","label"]].to_csv("./data/result/processed.csv", index=False)